User Profile Recommender System

Load Packages and Read in User Profile Simulation Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#df = pd.read_csv("user_profilesv2.csv")
df = pd.read_excel('profile_probs.xlsx')
df.shape
df.head(2)

(1000, 33)

,Male,Female,18 - 20,21 - 24,25 - 34,35 - 49,50 - 64,65+,Hispanic,Non_Hispanic,"< $7,500","$7,500 - $14,999","$15,000 - $24,999","$25,000 - $34,999","$35,000 - $49,999","$50,000 - $74,999","> $75,000",Never Married,Married,Widowed,Divorced,Separated,White,Black,Other,att,CRIM SEXUAL ASSAULT,ROBBERY,BATTERY,ASSAULT,BURGLARY,MOTOR VEHICLE THEFT,THEFT
0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,"['Male', '21 - 24', 'Hispanic', '$7,500 - $14,...",0.000004,0.000030,0.000009,0.000005,0.000013,0.000031,0.000009
1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,"['Female', '21 - 24', 'Hispanic', '< $7,500', ...",0.000443,0.000129,0.000494,0.000010,0.000043,0.000102,0.000064


In [ ]:
df = df.iloc[:,1:]

In [ ]:
features = df.loc[:,'att']
features

0      ['Male', '21 - 24', 'Hispanic', '$7,500 - $14,...
1      ['Female', '21 - 24', 'Hispanic', '< $7,500', ...
2      ['Male', '21 - 24', 'Hispanic', '$35,000 - $49...
3      ['Male', '35 - 49', 'Hispanic', '$25,000 - $34...
4      ['Female', '50 - 64', 'Non_Hispanic', '$7,500 ...
                             ...                        
995    ['Female', '21 - 24', 'Non_Hispanic', '$25,000...
996    ['Male', '18 - 20', 'Hispanic', '$7,500 - $14,...
997    ['Male', '35 - 49', 'Non_Hispanic', '$50,000 -...
998    ['Male', '25 - 34', 'Non_Hispanic', '$15,000 -...
999    ['Male', '50 - 64', 'Non_Hispanic', '< $7,500'...
Name: att, Length: 1000, dtype: object

Convert Attribute column into a count vector for all User Profiles to feed into Cosine Similarity Matrix

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['att'])

In [ ]:
count_matrix.toarray().shape
count_matrix.toarray()

(1000, 30)

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
cosine_sim = cosine_similarity(count_matrix)

In [ ]:
cosine_sim.shape
cosine_sim

(1000, 1000)

array([[1.        , 0.62994079, 0.52704628, ..., 0.40201513, 0.40201513,
        0.25197632],
       [0.62994079, 1.        , 0.47809144, ..., 0.11396058, 0.34188173,
        0.14285714],
       [0.52704628, 0.47809144, 1.        , ..., 0.47673129, 0.47673129,
        0.11952286],
       ...,
       [0.40201513, 0.11396058, 0.47673129, ..., 1.        , 0.54545455,
        0.34188173],
       [0.40201513, 0.34188173, 0.47673129, ..., 0.54545455, 1.        ,
        0.22792115],
       [0.25197632, 0.14285714, 0.11952286, ..., 0.34188173, 0.22792115,
        1.        ]])

Functions to get a user's index based on attribute and get a user's attributes based on index

In [ ]:
def get_user_from_index(index):
    return df[df.index == index]["att"].values[0]
def get_index_from_user(user):
    return df[df.att == user].index.values[0]

Recommendations based on similarity

In [ ]:
USER_NUM = 223

In [ ]:
user_att = df.iloc[USER_NUM].att
user_att

"['Male', '65+', 'Non_Hispanic', '< $7,500', 'Married', 'White']"

In [ ]:
user_index = get_index_from_user(user_att)
user_index

223

Sample recommender set

In [ ]:
similar_users = list(enumerate(cosine_sim[user_index])) #accessing the row corresponding to given user to find all the similarity scores for that user and then enumerating over it
sorted_similar_users = sorted(similar_users,key=lambda x:x[1],reverse=True)[1:]
i=0
print("Top 5 similar users to "+user_att+" are:\n")
for element in sorted_similar_users:
    print(get_user_from_index(element[0]))
    i=i+1
    if i>5:
        break

Top 5 similar users to ['Male', '65+', 'Non_Hispanic', '< $7,500', 'Married', 'White'] are:

['Male', '65+', 'Non_Hispanic', '< $7,500', 'Married', 'White']
['Male', '65+', 'Non_Hispanic', '< $7,500', 'Never Married', 'White']
['Male', '65+', 'Non_Hispanic', '< $7,500', 'Married', 'Other']
['Male', '65+', 'Non_Hispanic', '< $7,500', 'Married', 'Other']
['Male', '65+', 'Hispanic', '< $7,500', 'Never Married', 'White']
['Female', '65+', 'Non_Hispanic', '< $7,500', 'Never Married', 'White']


In [ ]:
sorted_similar_users[0:5]

[(957, 1.0000000000000002),
 (552, 0.9258200997725515),
 (818, 0.8333333333333336),
 (887, 0.8333333333333336),
 (62, 0.7715167498104596)]

In [ ]:
df.loc[552,'att']

"['Male', '65+', 'Non_Hispanic', '< $7,500', 'Never Married', 'White']"

Read in zip code file

In [ ]:
zip_codes = pd.read_csv('crimes_2015_to_present_with_zipcodes_enhanced.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Zip code preprocessing steps

In [ ]:
zip_na_rem = zip_codes[~zip_codes['zip'].isna()].reset_index(drop = True)

In [ ]:
zip_na_rem['zip'] = zip_na_rem['zip'].astype(int)

In [ ]:
zip_na_rem

,OID_,Join_Count,TARGET_FID,DateFormatted,ID,Case_Number,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,FBI_Code,X_Coordinate,Y_Coordinate,Year,Latitude,Longitude,Location,isDay,hour,IndexCode,isSerious,HATE_CRIME_DISPOSITION,HATE_CRIME_BIAS,HATE_CRIME_BIAS_CLASS,HATE_CRIME_BIAS_GROUP,HATE_CRIME_Location_Name,HATE_CRIME_Location_Code,HATE_CRIME_TYPE,isHateCrime,isHateCrimeDisposition,location_group,IsResidentialProperty,zipCode,objectid_1,zip
0,1,1,1,9/5/2015 13:30:00,10224738,HY411648,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,08B,1165074.0,1875917.0,2015.0,41.815117,-87.670000,"(41.815117282, -87.669999562)",Morning,13.0,NOT_FOUND,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,N,NaN,1.0,60609
1,2,1,2,9/4/2015 11:30:00,10224739,HY411615,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,06,1138875.0,1904869.0,2015.0,41.895080,-87.765400,"(41.895080471, -87.765400451)",Morning,11.0,NOT_FOUND,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,AIR/BUS/TRAIN TERMINAL,Y,NaN,2.0,60651
2,3,1,3,9/5/2015 12:45:00,10224740,HY411595,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,18,1152037.0,1920384.0,2015.0,41.937406,-87.716650,"(41.937405765, -87.716649687)",Morning,12.0,N,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,Y,NaN,3.0,60618
3,4,1,4,9/5/2015 13:00:00,10224741,HY411610,0560,ASSAULT,SIMPLE,APARTMENT,False,True,08A,1141706.0,1900086.0,2015.0,41.881903,-87.755121,"(41.881903443, -87.755121152)",Morning,13.0,NOT_FOUND,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,N,NaN,4.0,60644
4,5,1,5,9/5/2015 10:55:00,10224742,HY411435,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,1168430.0,1850165.0,2015.0,41.744379,-87.658431,"(41.744378879, -87.658430635)",Morning,10.0,NOT_FOUND,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,N,NaN,5.0,60620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77465,79018,1,79018,4/21/2016 10:00:00,10493859,HZ234424,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,14,1190596.0,1852000.0,2016.0,41.748909,-87.577154,"(41.748908514, -87.577153689)",Morning,10.0,N,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,N,NaN,79018.0,60617
77466,79019,1,79019,4/21/2016 11:54:00,10493862,HZ234541,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,True,False,18,1156710.0,1888914.0,2016.0,41.850956,-87.700329,"(41.850955693, -87.700328849)",Morning,11.0,N,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,HIGHWAY/ROAD/ALLEY/STREET,Y,NaN,79019.0,60623
77467,79020,1,79020,4/20/2016 13:30:00,10493864,HZ234600,0820,THEFT,$500 AND UNDER,HOSPITAL BUILDING/GROUNDS,False,False,06,1177877.0,1905128.0,2016.0,41.894993,-87.622149,"(41.894993403, -87.622148996)",Morning,13.0,NOT_FOUND,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,DRUG STORE/DOCTOR'S OFFICE/HOSPITAL,Y,NaN,79020.0,60611
77468,79021,1,79021,4/20/2016 12:00:00,10493868,HZ234254,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,05,1176538.0,1860994.0,2016.0,41.773917,-87.628397,"(41.773916563, -87.62839694)",Morning,12.0,NOT_FOUND,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,RESIDENCE/HOME,N,NaN,79021.0,60621


In [ ]:
zip_grouped = zip_na_rem.groupby(['zip','Primary_Type']).count().reset_index()
zip_pivot = zip_grouped.pivot_table('OID_',['zip'],'Primary_Type').reset_index().fillna(0)
zip_pivot

Primary_Type,zip,ARSON,ASSAULT,BATTERY,BURGLARY,CONCEALED CARRY LICENSE VIOLATION,CRIM SEXUAL ASSAULT,CRIMINAL DAMAGE,CRIMINAL TRESPASS,DECEPTIVE PRACTICE,GAMBLING,HUMAN TRAFFICKING,INTERFERENCE WITH PUBLIC OFFICER,INTIMIDATION,KIDNAPPING,LIQUOR LAW VIOLATION,MOTOR VEHICLE THEFT,NARCOTICS,NON - CRIMINAL,NON-CRIMINAL,OBSCENITY,OFFENSE INVOLVING CHILDREN,OTHER NARCOTIC VIOLATION,OTHER OFFENSE,PROSTITUTION,PUBLIC INDECENCY,PUBLIC PEACE VIOLATION,ROBBERY,SEX OFFENSE,STALKING,THEFT,WEAPONS VIOLATION
0,60601,0.0,29.0,34.0,10.0,0.0,0.0,36.0,16.0,112.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,4.0,15.0,2.0,0.0,349.0,0.0
1,60602,0.0,40.0,39.0,1.0,0.0,0.0,12.0,15.0,98.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,4.0,10.0,0.0,0.0,572.0,1.0
2,60603,1.0,5.0,22.0,1.0,0.0,0.0,12.0,17.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,0.0,1.0,0.0,16.0,0.0,0.0,2.0,8.0,0.0,0.0,299.0,1.0
3,60604,0.0,22.0,32.0,4.0,0.0,0.0,16.0,14.0,61.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,4.0,7.0,3.0,0.0,217.0,1.0
4,60605,0.0,45.0,66.0,15.0,0.0,2.0,44.0,41.0,143.0,0.0,0.0,1.0,0.0,2.0,1.0,21.0,18.0,0.0,0.0,0.0,2.0,0.0,41.0,0.0,0.0,7.0,22.0,6.0,1.0,397.0,1.0
5,60606,0.0,20.0,23.0,3.0,0.0,0.0,22.0,13.0,87.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,8.0,4.0,0.0,0.0,215.0,1.0
6,60607,1.0,58.0,78.0,44.0,0.0,3.0,66.0,29.0,124.0,0.0,0.0,1.0,2.0,0.0,0.0,28.0,18.0,1.0,0.0,0.0,6.0,0.0,81.0,0.0,0.0,5.0,9.0,5.0,0.0,436.0,1.0
7,60608,0.0,144.0,266.0,69.0,0.0,4.0,165.0,34.0,125.0,0.0,0.0,3.0,2.0,1.0,1.0,64.0,82.0,1.0,0.0,0.0,34.0,0.0,122.0,0.0,0.0,17.0,36.0,8.0,1.0,434.0,11.0
8,60609,2.0,190.0,395.0,165.0,0.0,4.0,196.0,50.0,153.0,1.0,0.0,15.0,2.0,3.0,0.0,78.0,146.0,0.0,0.0,0.0,21.0,0.0,196.0,4.0,0.0,11.0,77.0,12.0,1.0,471.0,39.0
9,60610,1.0,60.0,96.0,20.0,0.0,2.0,44.0,46.0,114.0,0.0,0.0,1.0,1.0,1.0,0.0,18.0,40.0,0.0,0.0,0.0,2.0,0.0,44.0,0.0,0.0,6.0,13.0,3.0,0.0,315.0,3.0


In [ ]:
df.columns

Index(['Female', '18 - 20', '21 - 24', '25 - 34', '35 - 49', '50 - 64', '65+',
       'Hispanic', 'Non_Hispanic', '< $7,500', '$7,500 - $14,999',
       '$15,000 - $24,999', '$25,000 - $34,999', '$35,000 - $49,999',
       '$50,000 - $74,999', '> $75,000', 'Never Married', 'Married', 'Widowed',
       'Divorced', 'Separated', 'White', 'Black', 'Other', 'att',
       'CRIM SEXUAL ASSAULT', 'ROBBERY', 'BATTERY', 'ASSAULT', 'BURGLARY',
       'MOTOR VEHICLE THEFT', 'THEFT'],
      dtype='object')

In [ ]:
zip_pivot = zip_pivot[['zip','CRIM SEXUAL ASSAULT','ROBBERY','BATTERY','ASSAULT','BURGLARY','MOTOR VEHICLE THEFT', 'THEFT']]
zip_pivot

Primary_Type,zip,CRIM SEXUAL ASSAULT,ROBBERY,BATTERY,ASSAULT,BURGLARY,MOTOR VEHICLE THEFT,THEFT
0,60601,0.0,15.0,34.0,29.0,10.0,6.0,349.0
1,60602,0.0,10.0,39.0,40.0,1.0,3.0,572.0
2,60603,0.0,8.0,22.0,5.0,1.0,3.0,299.0
3,60604,0.0,7.0,32.0,22.0,4.0,0.0,217.0
4,60605,2.0,22.0,66.0,45.0,15.0,21.0,397.0
5,60606,0.0,4.0,23.0,20.0,3.0,4.0,215.0
6,60607,3.0,9.0,78.0,58.0,44.0,28.0,436.0
7,60608,4.0,36.0,266.0,144.0,69.0,64.0,434.0
8,60609,4.0,77.0,395.0,190.0,165.0,78.0,471.0
9,60610,2.0,13.0,96.0,60.0,20.0,18.0,315.0


In [ ]:
'''def Fit_Dist(y,smooth = 0, tolerance = .05, plot = False):
  def Compute_RSS(y, smooth=0, distr = 'full'):
    ## If below cell errors remove hashtag from pip install distfit and re-run ##
    !pip install distfit
    from distfit import distfit
    if smooth != 0:
      dist = distfit(smooth = smooth, distr = distr)
    else:
      dist = distfit()
    dist.fit_transform(y)
    best_distr = dist.model;
    RSS = best_distr['score']
    return RSS, best_distr, dist

  #Based on the best theoretical distribution found, compute the cdf
  def DISTFIT_CDF(best_distr, y):
    name = best_distr['name']
    params = best_distr['params']
    model = best_distr['distr']
    print('locaton: ',params[-2])
    cdf = model.cdf(y, *params[:2])
    return cdf

  # Run the KS Test to see if y fits the selected distribution
  def KS_Test(best_distr, y):
    from scipy.stats import kstest as ks
    name = best_distr['name']
    params = best_distr['params']
    return ks(y, name, args = params)[1]

  #Compute the RSS and get the best distribution for y based on distfit function
  RSS, best_distr,dist = Compute_RSS(y, smooth = smooth)

  #check if the distribution and y pass the KS-test
  KS_pval = KS_Test(best_distr,y)
  # If passes, return CDF
  if (KS_pval > tolerance):
    print('smooth :', smooth)
    print('RSS ', RSS)
    print('mean: ', np.mean(y))
    print('parameters: ', best_distr['params'])
    cdf = DISTFIT_CDF(best_distr, y)

    if plot == True:
      dist.plot()
    return cdf

  #If not, smooth the dataset y twice
  elif smooth < 2:
    return Fit_Dist(y, smooth = smooth+1)
  
  #if they still fail, run the fitter() function to try out additonal distributions
  else:
    print('FITTER failed. KS_pval is: ', KS_pval, ' Additional methods need to be explored')
'''

In [ ]:
np.sort(zip_pivot.loc[:,'ASSAULT'])

array([  5.,  10.,  11.,  12.,  15.,  16.,  19.,  20.,  20.,  20.,  21.,
        22.,  23.,  25.,  29.,  29.,  31.,  31.,  33.,  34.,  35.,  39.,
        40.,  42.,  45.,  51.,  55.,  58.,  60.,  60.,  66.,  70.,  79.,
        83.,  83.,  90.,  90.,  93., 102., 105., 119., 127., 139., 144.,
       162., 172., 188., 190., 205., 206., 207., 218., 219., 220., 235.,
       263., 268., 304., 304.])

In [ ]:
## Apply Percentile Score Ranking
from scipy import stats
def zipcode_ranking(df,col):
  n = len(df)
  return [stats.percentileofscore(df.loc[:,col],df.loc[i,col])/100 for i in range(n)]

In [ ]:
ranking

[0.2627118644067796,
 0.3898305084745763,
 0.01694915254237288,
 0.20338983050847456,
 0.423728813559322,
 0.15254237288135594,
 0.4745762711864407,
 0.7457627118644068,
 0.8135593220338982,
 0.5,
 0.2627118644067796,
 0.7966101694915254,
 0.2966101694915254,
 0.3559322033898305,
 0.6186440677966102,
 0.6440677966101696,
 0.9661016949152542,
 0.5423728813559322,
 0.9491525423728814,
 0.9915254237288136,
 0.8305084745762712,
 0.5254237288135593,
 0.9152542372881356,
 0.8813559322033899,
 0.559322033898305,
 0.5,
 0.9915254237288136,
 0.7627118644067796,
 0.3728813559322034,
 0.08474576271186442,
 0.6610169491525424,
 0.10169491525423728,
 0.5847457627118644,
 0.8644067796610169,
 0.847457627118644,
 0.3220338983050848,
 0.711864406779661,
 0.6186440677966102,
 0.5847457627118644,
 0.2966101694915254,
 0.728813559322034,
 0.9322033898305085,
 0.22033898305084748,
 0.1864406779661017,
 0.6779661016949153,
 0.7796610169491526,
 0.8983050847457628,
 0.4576271186440678,
 0.6949152542372882,


Create zipcode ranking for each crime

In [ ]:
rank_vals = []
for col in zip_pivot.columns[1:]:
  rank = zipcode_ranking(zip_pivot, col)
  rank_vals.append(rank)

In [ ]:
len(rank_vals)

7

In [ ]:
rank_vals

[[0.1271186440677966,
  0.1271186440677966,
  0.1271186440677966,
  0.1271186440677966,
  0.4491525423728814,
  0.1271186440677966,
  0.5254237288135593,
  0.6186440677966102,
  0.6186440677966102,
  0.4491525423728814,
  0.3220338983050848,
  0.4491525423728814,
  0.3220338983050848,
  0.6186440677966102,
  0.1271186440677966,
  0.6186440677966102,
  0.9237288135593221,
  0.8559322033898304,
  0.8559322033898304,
  1.0,
  0.7796610169491526,
  0.4491525423728814,
  0.8559322033898304,
  0.9576271186440678,
  0.6186440677966102,
  0.711864406779661,
  0.8983050847457628,
  0.9237288135593221,
  0.3220338983050848,
  0.3220338983050848,
  0.8559322033898304,
  0.1271186440677966,
  0.4491525423728814,
  0.7796610169491526,
  0.711864406779661,
  0.1271186440677966,
  0.7796610169491526,
  0.9576271186440678,
  0.5254237288135593,
  0.1271186440677966,
  0.6186440677966102,
  0.7796610169491526,
  0.3220338983050848,
  0.1271186440677966,
  0.6186440677966102,
  0.9830508474576272,
  0.7

In [ ]:
dist = pd.concat([zip_pivot['zip'],pd.DataFrame(rank_vals, list(zip_pivot.columns[1:])).T], axis = 1)
dist

,zip,CRIM SEXUAL ASSAULT,ROBBERY,BATTERY,ASSAULT,BURGLARY,MOTOR VEHICLE THEFT,THEFT
0,60601,0.127119,0.423729,0.203390,0.262712,0.135593,0.177966,0.542373
1,60602,0.127119,0.313559,0.237288,0.389831,0.042373,0.059322,0.932203
2,60603,0.127119,0.254237,0.042373,0.016949,0.042373,0.059322,0.440678
3,60604,0.127119,0.220339,0.169492,0.203390,0.093220,0.016949,0.288136
4,60605,0.449153,0.533898,0.288136,0.423729,0.186441,0.398305,0.686441
5,60606,0.127119,0.110169,0.076271,0.152542,0.067797,0.118644,0.271186
6,60607,0.525424,0.271186,0.338983,0.474576,0.423729,0.474576,0.771186
7,60608,0.618644,0.677966,0.745763,0.745763,0.559322,0.745763,0.745763
8,60609,0.618644,0.805085,0.779661,0.813559,0.855932,0.796610,0.847458
9,60610,0.449153,0.381356,0.474576,0.500000,0.211864,0.305085,0.474576


In [ ]:
df.iloc[:,-len(rank_vals):]

,CRIM SEXUAL ASSAULT,ROBBERY,BATTERY,ASSAULT,BURGLARY,MOTOR VEHICLE THEFT,THEFT
0,3.961745e-06,0.000030,0.000009,0.000005,0.000013,0.000031,0.000009
1,4.428632e-04,0.000129,0.000494,0.000010,0.000043,0.000102,0.000064
2,9.006051e-05,0.000403,0.000054,0.000138,0.000084,0.000283,0.000113
3,2.705321e-07,0.000018,0.000005,0.000007,0.000006,0.000009,0.000008
4,8.016813e-04,0.000189,0.000402,0.000706,0.001116,0.000366,0.000396
...,...,...,...,...,...,...,...
995,5.256189e-03,0.001810,0.003903,0.001497,0.001939,0.001829,0.002075
996,3.186484e-06,0.000029,0.000004,0.000022,0.000003,0.000004,0.000002
997,2.130277e-04,0.001283,0.000161,0.001681,0.000762,0.000971,0.001247
998,9.392445e-04,0.006332,0.001277,0.006701,0.004412,0.006188,0.003280


Create scores for each user and each zipcode

Scores are calculated as the victim profile probability score divided by the percentile ranking for each zip code

In [ ]:
score_dict = {}
cols = list(dist.columns[1:])
for col in cols:
  zipcodes = []
  ind = []
  score = []
  for i in range(len(df)):
    for j in range(len(dist)):
      ind.append(i)
      score.append(df.loc[i,col]/dist.loc[j,col])
      zipcodes.append(dist.loc[j,'zip'])
  score_dict.update({col+'_score': score})

In [ ]:
scores_by_zip = pd.concat([pd.DataFrame({'ind':ind,'zip':zipcodes}),pd.DataFrame(score_dict)],axis = 1)

In [ ]:
df = df.merge(scores_by_zip, how = 'left', 
         left_on = df.index, 
         right_on = scores_by_zip.ind )

In [ ]:
score_cols = df.columns[df.columns.str.contains('score')]
for col in score_cols:
  score_col_mean = np.mean(df[col])
  df[col+'_count'] = np.where(df[col]> score_col_mean,1,0)

Create count column for each crime type by zip code by user
If a crime score is greater than or equal to the mean score for that crime score we add a value of 1 otherwise 0

In [ ]:
count_cols = list(df.columns[df.columns.str.contains('count')])
df['score_count'] = df[count_cols].sum(axis = 1)
df[df['score_count']>5]

,key_0,Male,Female,18 - 20,21 - 24,25 - 34,35 - 49,50 - 64,65+,Hispanic,Non_Hispanic,"< $7,500","$7,500 - $14,999","$15,000 - $24,999","$25,000 - $34,999","$35,000 - $49,999","$50,000 - $74,999","> $75,000",Never Married,Married,Widowed,Divorced,Separated,White,Black,Other,att,CRIM SEXUAL ASSAULT,ROBBERY,BATTERY,ASSAULT,BURGLARY,MOTOR VEHICLE THEFT,THEFT,ind,zip,CRIM SEXUAL ASSAULT_score,ROBBERY_score,BATTERY_score,ASSAULT_score,BURGLARY_score,MOTOR VEHICLE THEFT_score,THEFT_score,CRIM SEXUAL ASSAULT_score_count,ROBBERY_score_count,BATTERY_score_count,ASSAULT_score_count,BURGLARY_score_count,MOTOR VEHICLE THEFT_score_count,THEFT_score_count,score_count
265,4,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,"['Female', '50 - 64', 'Non_Hispanic', '$7,500 ...",0.000802,0.000189,0.000402,0.000706,0.001116,0.000366,0.000396,4,60631,0.002489,0.002478,0.005267,0.008328,0.005265,0.003085,0.005838,1,1,1,1,1,1,1,7
267,4,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,"['Female', '50 - 64', 'Non_Hispanic', '$7,500 ...",0.000802,0.000189,0.000402,0.000706,0.001116,0.000366,0.000396,4,60633,0.006307,0.005575,0.002633,0.006940,0.011967,0.003085,0.023351,1,1,1,1,1,1,1,7
279,4,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,"['Female', '50 - 64', 'Non_Hispanic', '$7,500 ...",0.000802,0.000189,0.000402,0.000706,0.001116,0.000366,0.000396,4,60646,0.006307,0.001239,0.003646,0.003785,0.003464,0.006170,0.007784,1,0,1,1,1,1,1,6
287,4,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,"['Female', '50 - 64', 'Non_Hispanic', '$7,500 ...",0.000802,0.000189,0.000402,0.000706,0.001116,0.000366,0.000396,4,60656,0.002489,0.002478,0.002155,0.004626,0.003989,0.002057,0.004670,1,1,1,1,1,1,1,7
291,4,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,"['Female', '50 - 64', 'Non_Hispanic', '$7,500 ...",0.000802,0.000189,0.000402,0.000706,0.001116,0.000366,0.000396,4,60661,0.006307,0.003717,0.023701,0.010409,0.009403,0.001661,0.002335,1,1,1,1,1,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58936,998,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,"['Male', '25 - 34', 'Non_Hispanic', '$15,000 -...",0.000939,0.006332,0.001277,0.006701,0.004412,0.006188,0.003280,998,60660,0.002917,0.028738,0.002844,0.019767,0.015777,0.024339,0.017591,1,1,1,1,1,1,1,7
58937,998,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,"['Male', '25 - 34', 'Non_Hispanic', '$15,000 -...",0.000939,0.006332,0.001277,0.006701,0.004412,0.006188,0.003280,998,60661,0.007389,0.124530,0.075371,0.098833,0.037189,0.028084,0.019350,1,1,1,1,1,1,1,7
58938,998,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,"['Male', '25 - 34', 'Non_Hispanic', '$15,000 -...",0.000939,0.006332,0.001277,0.006701,0.004412,0.006188,0.003280,998,60666,0.007389,0.373591,0.030148,0.131778,0.260325,0.022818,0.021500,1,1,1,1,1,1,1,7
58939,998,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,"['Male', '25 - 34', 'Non_Hispanic', '$15,000 -...",0.000939,0.006332,0.001277,0.006701,0.004412,0.006188,0.003280,998,60707,0.002917,0.041510,0.009421,0.056476,0.018595,0.028084,0.027643,1,1,1,1,1,1,1,7


Define parameters for user ratings

In [ ]:
df['user_rating'] = list(map(lambda x:
5 if x < 2
else 4 if x < 4
else 3 if x < 5
else 2 if x < 6
else 1
,df['score_count']))


In [ ]:
def get_similar_users(user_index):
  similar_users = list(enumerate(cosine_sim[user_index])) #accessing the row corresponding to given user to find all the similarity scores for that user and then enumerating over it
  sorted_similar_users = sorted(similar_users,key=lambda x:x[1],reverse=True)[1:]
  user_att = df.iloc[user_index].att
  i=1
  print("Top 5 similar users to user "+user_att+" are:\n")
  for element in sorted_similar_users:
      print(get_user_from_index(element[0]))
      i=i+1
      if i>5:
          break
  return sorted_similar_users

In [ ]:
sorted_similar_users = get_similar_users(223)

Top 5 similar users to user ['Male', '35 - 49', 'Hispanic', '$25,000 - $34,999', 'Widowed', 'Other'] are:

['Male', '25 - 34', 'Non_Hispanic', '$25,000 - $34,999', 'Separated', 'Other']
['Male', '65+', 'Non_Hispanic', '$25,000 - $34,999', 'Widowed', 'Black']
['Male', '50 - 64', 'Hispanic', '> $75,000', 'Widowed', 'Other']
['Female', '65+', 'Hispanic', '$50,000 - $74,999', 'Married', 'White']
['Female', '21 - 24', 'Hispanic', '< $7,500', 'Married', 'White']


In [ ]:
def get_similar_user_rating(user_index,sorted_similar_users):
  most_similar = sorted_similar_users[0][0]
  similar_df = df[df.ind == most_similar][['att','zip','user_rating']]
  similar_df.insert(0,'user_index',user_index)
  for su in sorted_similar_users[1:5]:
    temp = df[df.ind == su[0]][['zip','user_rating']]
    similar_df = similar_df.merge(temp, how = 'inner', on = 'zip')
  similar_df.insert(len(similar_df.columns),'mean_rating',similar_df.iloc[:,3:].mean(axis = 1 ))

  return similar_df

In [ ]:
get_similar_user_rating(223,sorted_similar_users)

,user_index,att,zip,user_rating_x,user_rating_y,user_rating_x,user_rating_y,user_rating,mean_rating
0,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60601,5,5,5,5,4,4.8
1,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60602,5,5,5,4,3,4.4
2,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60603,5,4,4,3,1,3.4
3,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60604,5,5,5,4,2,4.2
4,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60605,5,5,5,5,5,5.0
5,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60606,5,5,5,3,2,4.0
6,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60607,5,5,5,5,5,5.0
7,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60608,5,5,5,5,5,5.0
8,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60609,5,5,5,5,5,5.0
9,223,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60610,5,5,5,5,5,5.0


Get user recommendations for 10 randomly chosen users

In [ ]:
random_ten_df = pd.DataFrame(columns = similar_df.columns)
np.random.seed(223)
random_users = np.random.randint(0,len(cosine_sim),10)
for user in random_users:
  sorted_similar_users = get_similar_users(user)
  similar_df = get_similar_user_rating(user,sorted_similar_users)
  random_ten_df = pd.concat([random_ten_df,similar_df], axis = 0)

Top 5 similar users to user ['Female', '65+', 'Hispanic', '> $75,000', 'Widowed', 'White'] are:

['Female', '65+', 'Hispanic', '> $75,000', 'Widowed', 'White']
['Male', '35 - 49', 'Non_Hispanic', '> $75,000', 'Married', 'Black']
['Male', '50 - 64', 'Non_Hispanic', '$7,500 - $14,999', 'Separated', 'White']
['Female', '65+', 'Hispanic', '> $75,000', 'Widowed', 'White']
['Male', '50 - 64', 'Non_Hispanic', '$7,500 - $14,999', 'Separated', 'White']
Top 5 similar users to user ['Male', '21 - 24', 'Hispanic', '$7,500 - $14,999', 'Married', 'Other'] are:

['Female', '65+', 'Hispanic', '$50,000 - $74,999', 'Married', 'White']
['Female', '65+', 'Hispanic', '$50,000 - $74,999', 'Married', 'White']
['Male', '21 - 24', 'Hispanic', '$7,500 - $14,999', 'Married', 'Other']
['Male', '50 - 64', 'Non_Hispanic', '$7,500 - $14,999', 'Separated', 'White']
['Male', '65+', 'Non_Hispanic', '$25,000 - $34,999', 'Widowed', 'Black']
Top 5 similar users to user ['Female', '35 - 49', 'Non_Hispanic', '$25,000 - $34,

In [ ]:
random_ten_df.reset_index(drop = True, inplace = True)
random_ten_df

,user_index,att,zip,user_rating_x,user_rating_y,user_rating_x,user_rating_y,user_rating,mean_rating
0,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60601,1,4,4,1,4,2.8
1,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60602,1,4,4,1,4,2.8
2,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60603,1,3,2,1,2,1.8
3,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60604,1,4,2,1,2,2.0
4,478,"['Male', '35 - 49', 'Hispanic', '> $75,000', '...",60605,1,5,5,1,4,3.2
...,...,...,...,...,...,...,...,...,...
585,204,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60660,5,5,5,5,4,4.8
586,204,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60661,5,4,4,4,2,3.8
587,204,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60666,4,4,4,4,1,3.4
588,204,"['Female', '35 - 49', 'Hispanic', '$25,000 - $...",60707,5,5,5,4,3,4.4


In [ ]:
random_ten_df.to_csv('ten_user_recommendations.csv')